# создаём чат_бота

In [7]:
# это код самого телеграмма. Его не трогаем
import logging

from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

# Enable logging
#logging.basicConfig(
#    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
#)

#logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def get_message(update: Update, context: CallbackContext) -> None:
    """Get the user message."""
    print('Входящее сообщение update.message.text:', update.message.text)
    # точка входа: сообщение из телеграмма update.message.text 
    # здесь попадает в нашу программу
    
    try:
        answer = update.message.text
        otvet = my_func(answer) # наша программа генерит ответ answer        
        
    except:
        otvet = "Что-то пошло не так ;("
    
    # ответ answer улетает в телеграмм
    try:        
        update.message.reply_text(otvet) #отправка ответа
    except:
        answer = "Что-то сломалось. Извините ;("
        update.message.reply_text(answer)


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("2130329456:AAGRRUymCdLRgfm40dOBgOIyNJdwzDjRN10")

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - get_message the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, get_message))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()



In [16]:
# вопросник для условных вероятностей свидетельств
import time
import random
import copy

list_quest = ['Как часто может быть такое, что ',
              'Как часто Вы сталкивались с такой ситуацией, что ',
              'Какова вероятность того, что ',
              'Какова вероятность ситуации, что ',
              'В скольких ситуациях из 100 возможно, что ',
              'Как часто Может быть так, что ',
              'Как часто бывает так, что ',
              'Какова может быть доля ситуаций таких, что' ]
lst_condits = [[' при условии, что ', 'в ситуации, что ', 'и одновременно '],
               [' при условии, что НЕ ВЫПОЛНЕНО: ',' при том, что НЕ ВЕРНО, что ',' вместе с тем НЕ ВЕРНО, что ']]

H = ['Сел аккумулятор','Нет бензина', 'Отсырел распределитель зажигания','Загрязнены свечи']
    
# H = {0:[0.1, 'Сел аккумулятор',                 [[1e-10,0.99],[0.7,0.05],[0.0,0.0],[0.2,0.5], [1e-10,0.99],[1,0.01]]],
#      1:[0.05,'Нет бензина',                     [[0,0], [1,0.01],  [0.0,0.0],[0.0,0.0], [0,0.0], [0.9,0.02]]],
#      2:[0.01,'Отсырел распределитель зажигания',[[0,0],   [1e-10,0.01],  [0.9,0.1],[0.25,0.5],[0,0.0], [0.9,0.02] ]],
#      3:[0.01,'Загрязнены свечи',                [[0,0],   [0,0],     [0,0],    [0.01,0.5],[0,0],   [0.9,0.02]]],    
#     }

E = {0:('Фары горят','Горят ли фары?'),
     1:('Указатель бензина на нуле','Есть ли бензин?'),
     2:('Автомобиль отсырел','Не находился ли автомобиль долго под дождем?'),
     3:('Автомобиль недавно прошел ТО','Проходил ли автомобиль недавно ТО?'),
     4:('Стартер крутится','Крутится ли стартер?'),
     5:('Автомобиль не заводится','Автомобиль не заводится?'),
    }

# создали шаблон БД гипотез
H_DB = {j:[str(i)+str(j) for i in range(3)] for j in range(len(H))}
for i in H_DB:
    H_DB[i][2] = [[0,0] for j in range(len(E))]
    

ijk = [0,0,0]

#=====================================функции=====================================================================

def my_func(text):
    global ijk, H_DB
    
    print('text=',text)
    print('ijk =', ijk)
    print(H_DB[ijk[0]][2][ijk[1]][ijk[2]])
    try:        
        probab = float(text)
        if H_DB[ijk[0]][2][ijk[1]][ijk[2]] == 0:
            if probab == 0:
                H_DB[ijk[0]][2][ijk[1]][ijk[2]] = 1e-10
            elif probab > 1:
                #H_DB[i][2][j][k] = 1
                H_DB[ijk[0]][2][ijk[1]][ijk[2]] = 1
            else:            
                #H_DB[i][2][j][k] = text
                H_DB[ijk[0]][2][ijk[1]][ijk[2]] = probab
            print("новый элемент H_DB: ",H_DB[ijk[0]][2][ijk[1]][ijk[2]])    
            print(H_DB)    
    except:
        pass
        
    res = create_question(list_quest,lst_condits,H,E)
    ijk = list(res[1:])
    print('res=',res)
    print('ijk =', ijk)
    return res[0]

def create_question(list_quest,lst_condits,H,E):
    """
    Функция генерит тройку чисел i,j,k. 
    Если H[i][2][j][k] ==0, тогда 
    на основе входных списков гипотез и свидетельств(симптомов)
    для пары (E_j,H_i) задаёт вопросы для выяснения у эксперта значения
    k-й (=0,1) условной вероятности свидетельства: P0(E_j/H_i) или P1(E_j/~H_i)    
    """ 
    global H_DB
        
    i = random.randint(0,len(H)-1)
    j = random.randint(0,len(E)-1)
    k = random.randint(0,1)
    
    start = time.time()  # время начала выполнения
    while H_DB[i][2][j][k] != 0:
        i = random.randint(0,len(H)-1)
        j = random.randint(0,len(E)-1)
        k = random.randint(0,1)
        if time.time() - start > 5:
            return "Вопросов больше не осталось!!!",0,0,0
        
    return random.choice(list_quest)+'"'+E[j][0]+'"'+random.choice(lst_condits[k])+ H[i],i,j,k

In [17]:
H_DB = {0: ['00', '10', [[1e-10, 0.876], [0.11111, 0.77777], [0.8643, 0.8888888888], [0.434343, 0.212], [0, 1e-10], [0.422, 0.676767]]], 1: ['01', '11', [[0.7543, 0.875], [0.888888, 0.64333], [1e-10, 0.6553322], [0.4443, 0.46], [0.22222, 6.43e-05], [0.84321, 0.43]]], 2: ['02', '12', [[0.9, 0.644], [0.479, 0.223], [0.5, 0.54], [0.64322, 0.555555], [0.3214, 0.8532], [0.333, 0.999999]]], 3: ['03', '13', [[0.653222, 0.6543], [0.67, 0.754], [0.231478, 0.954367], [0.2223, 0.5326], [0.8, 0.8888], [0.5433356, 0.9533]]]}
H_DB

{0: ['00',
  '10',
  [[1e-10, 0.876],
   [0.11111, 0.77777],
   [0.8643, 0.8888888888],
   [0.434343, 0.212],
   [0, 1e-10],
   [0.422, 0.676767]]],
 1: ['01',
  '11',
  [[0.7543, 0.875],
   [0.888888, 0.64333],
   [1e-10, 0.6553322],
   [0.4443, 0.46],
   [0.22222, 6.43e-05],
   [0.84321, 0.43]]],
 2: ['02',
  '12',
  [[0.9, 0.644],
   [0.479, 0.223],
   [0.5, 0.54],
   [0.64322, 0.555555],
   [0.3214, 0.8532],
   [0.333, 0.999999]]],
 3: ['03',
  '13',
  [[0.653222, 0.6543],
   [0.67, 0.754],
   [0.231478, 0.954367],
   [0.2223, 0.5326],
   [0.8, 0.8888],
   [0.5433356, 0.9533]]]}

In [18]:
# запускаем чат_бота
main()

Входящее сообщение update.message.text: Врвр
text= Врвр
ijk = [0, 0, 0]
1e-10
res= ('Как часто Вы сталкивались с такой ситуацией, что "Стартер крутится"в ситуации, что Сел аккумулятор', 0, 4, 0)
ijk = [0, 4, 0]
Входящее сообщение update.message.text: .43
text= .43
ijk = [0, 4, 0]
0
новый элемент H_DB:  0.43
{0: ['00', '10', [[1e-10, 0.876], [0.11111, 0.77777], [0.8643, 0.8888888888], [0.434343, 0.212], [0.43, 1e-10], [0.422, 0.676767]]], 1: ['01', '11', [[0.7543, 0.875], [0.888888, 0.64333], [1e-10, 0.6553322], [0.4443, 0.46], [0.22222, 6.43e-05], [0.84321, 0.43]]], 2: ['02', '12', [[0.9, 0.644], [0.479, 0.223], [0.5, 0.54], [0.64322, 0.555555], [0.3214, 0.8532], [0.333, 0.999999]]], 3: ['03', '13', [[0.653222, 0.6543], [0.67, 0.754], [0.231478, 0.954367], [0.2223, 0.5326], [0.8, 0.8888], [0.5433356, 0.9533]]]}
res= ('Вопросов больше не осталось!!!', 0, 0, 0)
ijk = [0, 0, 0]
